In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def compute_loss(model,feature,label,batch,opt,train,loss_func,device): 
    total_loss=0
    start=0
    for j in range(int(len(feature)/batch)+1):
        if start+batch>len(feature):
            end=-1
        else:
            end=start+batch
            
        x=feature[start:end].to(device) #x,y放到gpu裡
        y=label[start:end].to(device)
        out=model(x)
        out=out.to(device)
        loss=loss_func(out,y)
        if train:
            opt.zero_grad() #梯度歸零
            loss.backward() #back propogation
            opt.step() #更新參數
        #print('batch loss:'+str(loss.item()))
        total_loss+=loss.item()
        torch.cuda.empty_cache()
        start=end
    total_loss=total_loss/(int(len(feature)/batch)+1)
    return total_loss

def mape(prediction,truth,M,m):
    diff=(prediction-truth).detach().numpy()*M
    diff=np.absolute(diff/(truth.detach().numpy()*M+m))
    return diff.mean()


def mae(prediction, truth):
    diff=np.absolute((prediction-truth).detach().numpy())
    return diff.mean()


def plot_result(xs,ys,output_len,title,target_series):
    result=[]
    truth=[]
    for i in range(2):
        for j in range(0,xs[i].shape[0],output_len):
            prediction=model(xs[i][j].unsqueeze(0))[:,:,0].squeeze(0).tolist()
            y=ys[i][j,:,0].tolist()
        
            result.extend([k*(scale[0][target_series]-scale[1][target_series])+scale[1][target_series] for k in prediction])
            truth.extend([k*(scale[0][target_series]-scale[1][target_series])+scale[1][target_series] for k in y])
        
    plt.plot(truth)
    plt.plot(result,alpha=0.5)
    plt.legend(['truth','prediction'])
    plt.title(title)
    #plt.savefig(title+'.png')
    plt.show()
    plt.clf()
    return result,truth
        
        


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [ ]:
    
def train(target, epochs, input_dim, input_len, output_len, split, model,constrain=False):
    testing_mae=0
    testing_mape=0
    
    #for i in targets:
    print(data.columns[target])
    rank=np.argsort(corr[:,i])[::-1]
    
    dataset1,dataset2=make_io(input_len,output_len,rank[0:input_dim],split)
    dataset3,dataset4=make_io(input_len,output_len,rank[0:input_dim],split,False)

    X_train=torch.FloatTensor(dataset1[0])
    Y_train=torch.FloatTensor(dataset1[1])[:,:,0].unsqueeze(2)
    S_train=torch.FloatTensor(dataset1[2])

    X_test=torch.FloatTensor(dataset2[0])
    Y_test=torch.FloatTensor(dataset2[1])[:,:,0].unsqueeze(2)
    S_test=torch.FloatTensor(dataset2[2])

    X_train_valid=torch.FloatTensor(dataset3[0])
    Y_train_valid=torch.FloatTensor(dataset3[1])[:,:,0].unsqueeze(2)
    S_train_valid=torch.FloatTensor(dataset3[2])

    X_test_valid=torch.FloatTensor(dataset4[0])
    Y_test_valid=torch.FloatTensor(dataset4[1])[:,:,0].unsqueeze(2)
    S_test_valid=torch.FloatTensor(dataset4[2])
        
    model.to(gpu) #model放到gpu裡
    print(count_parameters(model))
    
    for j in range(epochs):
        if j<500:
            rate=0.001
        elif j>500 and j<2000:
            rate=0.0001
        else:
            rate=0.00001
            
        opt=torch.optim.Adam(model.parameters(),lr=rate)
        
        if constrain:
            model.weight.data.clamp_(0,1)
   
        training_loss=compute_loss(model,X_train,Y_train,batch,opt,train=True,device=gpu)
        testing_loss=compute_loss(model,X_test,Y_test,batch,opt,train=False,device=gpu)
        test_mape=mape(model(X_test.to(gpu))[:,:,0].to(cpu),Y_test[:,:,0],(scale[0][i]-scale[1][i]),scale[1][i])
        
        if j%100==0:
            print('epoch:'+str(j))
            print('training mae loss:'+str(training_loss))
            print('testing mae loss:'+str(testing_loss))
            print('testing mape:'+str(test_mape))
            if constrain:
                print(model.weight.data)
            
    model.to(cpu)        
    #    testing_mae+=testing_loss
    #    testing_mape+=mape(model(X_test)[:,:,0],Y_test[:,:,0],(scale[0][i]-scale[1][i]),scale[1][i])
    plot_result([X_train_valid,X_test_valid],[Y_train_valid,Y_test_valid],7,data.columns[i],i)
    
    return X_train_valid, X_test_valid, rank
    #print('testing mae:')
    #print(testing_mae/len(targets))
    #print('testing mape:')
    #print(testing_mape/len(targets))

In [ ]:
def plot_attention(model, X_train, X_test, title_name):
    weights=[]
    for i in range(len(X_train)):
        _,_,sample=model.encoder(X_train[i].unsqueeze(0))
        weights.append(sample[0][0].tolist())
    
    for i in range(len(X_test)):
        _,_,sample=model.encoder(X_test[i].unsqueeze(0))
        weights.append(sample[0][0].tolist())    
        
    w=np.array(weights).transpose()
    w.shape
    plt.pcolor(w)
    plt.yticks([i for i in range(6)],[data.columns[i] for i in rank[1:]])
    plt.title(title_name)
    